In [6]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

from scipy.stats import norm
from scipy.optimize import brentq

import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
obs_day_1 = pd.read_csv('./p3_data/round4/observations_round_4_day_1.csv', sep=',')
obs_day_2 = pd.read_csv('./p3_data/round4/observations_round_4_day_2.csv', sep=',')
obs_day_3 = pd.read_csv('./p3_data/round4/observations_round_4_day_3.csv', sep=',')

prices_day_1 = pd.read_csv('./p3_data/round4/prices_round_4_day_1.csv', sep=';')
prices_day_2 = pd.read_csv('./p3_data/round4/prices_round_4_day_2.csv', sep=';')
prices_day_3 = pd.read_csv('./p3_data/round4/prices_round_4_day_3.csv', sep=';')

obs_day_2['timestamp'] += 1_000_000
obs_day_3['timestamp'] += 2_000_000

prices_day_2['timestamp'] += 1_000_000
prices_day_3['timestamp'] += 2_000_000

prices_day_1 = prices_day_1[prices_day_1['product'] == "MAGNIFICENT_MACARONS"]
prices_day_2 = prices_day_2[prices_day_2['product'] == "MAGNIFICENT_MACARONS"]
prices_day_3 = prices_day_3[prices_day_3['product'] == "MAGNIFICENT_MACARONS"]

obs = pd.concat([obs_day_1, obs_day_2, obs_day_3])
prices = pd.concat([prices_day_1, prices_day_2, prices_day_3])

obs_renamed = obs.rename(columns={
    'bidPrice': 'pristineBidPrice',
    'askPrice': 'pristineAskPrice'
})

prices_selected = prices[['timestamp', 'bid_price_1', 'ask_price_1']].rename(columns={
    'bid_price_1': 'archipelagoBidPrice',
    'ask_price_1': 'archipelagoAskPrice'
})

df = pd.merge(obs_renamed, prices_selected, on='timestamp')
df['conversionBuyCost'] = df['pristineAskPrice'] + df['transportFees'] + df['importTariff']
df['conversionSellRevenue'] = df['pristineBidPrice'] - df['transportFees'] - df['importTariff']
df['midPrice'] = (df['archipelagoBidPrice'] + df['archipelagoAskPrice']) / 2
print(df[df['timestamp'] == 2_000])

    timestamp  pristineBidPrice  pristineAskPrice  transportFees  \
20       2000             644.0             645.5            1.0   

    exportTariff  importTariff  sugarPrice  sunlightIndex  \
20           9.5          -3.0  201.093455          60.05   

    archipelagoBidPrice  archipelagoAskPrice  conversionBuyCost  \
20                644.0                  649              643.5   

    conversionSellRevenue  midPrice  
20                  646.0     646.5  


In [26]:
test = df['archipelagoBidPrice'] - df['conversionBuyCost']
print(test[test > 0])
print(sum(test[test > 0]))

test = df['conversionSellRevenue'] - df['archipelagoAskPrice']
print(test[test > 0])
print(sum(test[test > 0]))


20       0.5
100      1.0
186      1.0
225      2.0
282      1.0
        ... 
29929    2.8
29932    2.8
29951    2.8
29981    2.3
29983    3.8
Length: 782, dtype: float64
1289.2
54       0.5
82       0.5
95       1.0
156      2.5
163      1.0
        ... 
29778    2.3
29811    2.8
29915    2.3
29967    2.8
29993    3.8
Length: 809, dtype: float64
1317.9000000000012
